<a href="https://colab.research.google.com/github/Aggregate-Intellect/xir/blob/main/manually_entity_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install tensorflow
!pip install qwikidata
!pip install pywikibot
!pip install mwparserfromhell

In [2]:
import pandas as pd
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever, EmbeddingRetriever
from haystack.utils import fetch_archive_from_http, print_answers, print_documents
from haystack.pipelines import DocumentSearchPipeline
from haystack.document_stores import InMemoryDocumentStore

from datasets import load_dataset

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

import nltk.data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
vali_wikiqa_data = load_dataset(
    'wiki_qa',
    split='test',
)
pd_test_data = pd.DataFrame(vali_wikiqa_data)

pd_test_data = pd.DataFrame(vali_wikiqa_data)

# make sure all sentences ending with .
# if last character is not . ? !, add . at the end
for index in range(0,pd_test_data.shape[0]):
  last_char = pd_test_data['answer'][index][-1]
  if not (last_char =='.' or last_char =='!' or last_char =='?'):
    pd_test_data['answer'][index] = pd_test_data['answer'][index] + '.'


processed_test_wikiqa = pd.DataFrame(columns = ["question_id", "question", "document_title", "document_content", "answer"])
processed_all_wikiqa = pd.DataFrame(columns = ["question_id", "document_title", "document_content"])

for question_id in pd_test_data['question_id'].unique().tolist():
    # print(question_id)
    pd_question = pd_test_data.loc[pd_test_data['question_id'] == question_id]   
    # print(pd_question)
    content = ' '.join(pd_question["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1]
    # print(pd_question["document_title"].iloc[0])
    processed_all_wikiqa = processed_all_wikiqa.append({"question_id": question_id,
                                                        "document_title": pd_question["document_title"].iloc[0],
                                                        "document_content" : content}, ignore_index=True)
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        new_row = {"question_id": pd_question["question_id"].iloc[0], 
                  "question" : pd_question["question"].iloc[0],
                  "document_title" : pd_question["document_title"].iloc[0],
                  "document_content" : content,
                  "answer": answer}
        processed_test_wikiqa = processed_test_wikiqa.append(new_row, ignore_index=True)
processed_test_wikiqa

Generating test split:   0%|          | 0/6165 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2733 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20360 [00:00<?, ? examples/s]

Dataset wiki_qa downloaded and prepared to /root/.cache/huggingface/datasets/wiki_qa/default/0.1.0/d2d236b5cbdc6fbdab45d168b4d678a002e06ddea3525733a24558150585951c. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,question_id,question,document_title,document_content,answer
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States,African immigration to the United States refers to immigrants to the United ...,"As such, African immigrants are to be distinguished from African American pe..."
1,Q4,how a water pump works,Pump,"A small, electrically powered pump. A large, electrically driven pump (elect...","Pumps operate by some mechanism (typically reciprocating or rotary ), and co..."
2,Q20,how old was sue lyon when she made lolita,Lolita (1962 film),Lolita is a 1962 comedy-drama film by Stanley Kubrick based on the classic n...,"The actress who played Lolita, Sue Lyon , was fourteen at the time of filming."
3,Q33,how are antibodies used in,antibody,Each antibody binds to a specific antigen ; an interaction similar to a lock...,"An antibody (Ab), also known as an immunoglobulin (Ig), is a large Y-shaped ..."
4,Q59,HOW MUCH IS CENTAVOS IN MEXICO,Mexican peso,The peso ( sign : $; code : MXN) is the currency of Mexico . Modern peso and...,"The peso is subdivided into 100 centavos, represented by "" ¢ ""."
...,...,...,...,...,...
238,Q2990,what is the main component of vaccines,vaccine,Jonas Salk in 1955 holds two bottles of a culture used to grow polio vaccine...,The agent stimulates the body's immune system to recognize the agent as fore...
239,Q2994,what is preciosa crystal?,Preciosa (corporation),Preciosa is the luxury brand name for the range of precision-cut lead crysta...,Preciosa is the luxury brand name for the range of precision-cut lead crysta...
240,Q3004,who are all of the jonas brothers,Jonas Brothers,"The Jonas Brothers are an American pop rock band. Formed in 2005, they have ...","Formed in 2005, they have gained popularity from the Disney Channel children..."
241,Q3008,who is mary matalin married to,Mary Matalin,"This is about the political professional. For the actress, see Marlee Matlin...",She is married to Democratic political consultant James Carville .


In [6]:
import pandas as pd

pd_test_data = pd.read_csv("/content/entity&relationship_extraction.csv")

In [7]:
#df_direct_answer = pd_test_data[pd_test_data['answer by entities and relationships'].notna()]
cols = ['answer by entities and relationships', 'entities']
df_direct_answer = pd_test_data[pd_test_data[cols].notnull().any(axis=1)]
df_direct_answer

,question,answer by entities and relationships,entities_name,entities,relationships
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,NaN,"African_Americans, immigration, United_States","Q49085, Q131288, Q30",[]
1,how a water pump works,NaN,"Water pump, work","Q2245442, Q42213",[]
2,how old was sue lyon when she made lolita,NaN,"Sue Lyon, Lolita","Q436722, Q536560",[]
3,how are antibodies used in,NaN,"antibody, use","Q79460, Q1724915",[]
4,HOW MUCH IS CENTAVOS IN MEXICO,NaN,Mexican centavo,Q4730,[]
...,...,...,...,...,...
238,what is the main component of vaccines,NaN,vaccine,Q134808,[]
239,what is preciosa crystal?,NaN,Presciosa,Q3502967,[]
240,who are all of the jonas brothers,"Kevin Jonas, Joe Jonas, Nick Jonas",NaN,Q134233,"[['has parts', 'P527']]"
241,who is mary matalin married to,James Carville,NaN,Mary Matalin (American political consultant) / spouse,NaN


# query expansion using knowledge graph

In [8]:
def query_expansion(original_query):
  pd_question = df_direct_answer.loc[df_direct_answer['question'] == original_query] 
  
  if not pd_question.empty: 
    
    # first
    if str(pd_question['answer by entities and relationships'].iloc[0]) != 'nan':
      new_query = original_query + " " + pd_question['answer by entities and relationships'].iloc[0]
      return new_query
    
    # second
    elif str(pd_question['entities'].iloc[0]) != 'nan' and len(pd_question['entities'].iloc[0].split(", ")) == 1:
      # entity_id_list = pd_question['entities'].iloc[0].split(", ")
      entity_id = pd_question['entities'].iloc[0]
      entity_dict = get_entity_dict_from_api(entity_id)
      entity_dict = WikidataItem(entity_dict)
      description = entity_dict.get_description()
      new_query = original_query + " " + description
      return new_query
    
    else:
      if str(pd_question['entities_name'].iloc[0]) != 'nan':
        new_query = original_query + str(pd_question['entities_name'].iloc[0])
        return new_query
      
  return original_query

# Calculate accuracy

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_total = 0
number_correct = 0

for index, row in processed_test_wikiqa.iterrows():
  query = row['question']
  new_query = query_expansion(query)
  if query != new_query:
    number_total += 1
    print("Original Query: ", query)
    print("New Query: ", new_query)
    sentence_list = nltk.tokenize.sent_tokenize(row['document_content'])
    
    sentence_document_store = InMemoryDocumentStore()
    sentence_document_store.delete_documents()
    
    docs = []
    for sentence in sentence_list:
      doc = Document(content=sentence,
                     meta={"article_title": row['document_title']})
      docs.append(doc)
    sentence_document_store.write_documents(docs)
    
    sentence_retriever = EmbeddingRetriever(
          document_store=sentence_document_store,
          embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
          model_format="sentence_transformers")
    
    sentence_document_store.update_embeddings(sentence_retriever)
    sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
    sentence_result = sentence_pipeline.run(query=new_query, 
                                            params={"Retriever": {"top_k": 1}})
    retrieved_sentence = sentence_result["documents"][0].content
    target_sentence = processed_test_wikiqa['answer'][index]
    if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
        number_correct += 1
        print("!!!!!!!!!")
        print("Found")
    else:
        print("Not Found")

In [10]:
print(number_total)
print(number_correct / number_total)

101
0.7425742574257426
